# Embryo Data Query

This notebook queries embryo-related data from multiple databases for a given prontuario/PIN.

## Tables Queried:
- **Clinisys Silver Layer** (clinisys_all.duckdb):
  - view_micromanipulacao
  - view_micromanipulacao_oocitos
  - view_tratamentos
  - view_congelamentos_embrioes
  - view_descongelamentos_embrioes
  - view_embrioes_congelados
- **Huntington Data Lake** (huntington_data_lake.duckdb):
  - gold.clinisys_embrioes
  - gold.embryoscope_embrioes
  - gold.embryoscope_clinisys_combined
  - gold.embryoscope_treatment (from embryoscope_embrioes)
  - silver.planilha_embriologia
  - gold.planilha_embryoscope_combined


In [1]:
import duckdb
import pandas as pd
import os
from datetime import datetime

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


## Database Connections


In [2]:
def get_clinisys_connection():
    """Get connection to clinisys_all database"""
    clinisys_db_path = os.path.join('..', '..', 'database', 'clinisys_all.duckdb')
    return duckdb.connect(clinisys_db_path, read_only=True)

def get_huntington_connection():
    """Get connection to huntington_data_lake database"""
    huntington_db_path = os.path.join('..', '..', 'database', 'huntington_data_lake.duckdb')
    return duckdb.connect(huntington_db_path, read_only=True)


## Query Functions - Clinisys Tables


In [3]:
def query_micromanipulacao(conn, prontuario):
    """Query view_micromanipulacao for a specific prontuario"""
    print(f"\n=== VIEW_MICROMANIPULACAO ===")
    query = f"""
    SELECT * FROM silver.view_micromanipulacao 
    WHERE prontuario = {prontuario}
    ORDER BY codigo_ficha DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_micromanipulacao")
    return df

def query_micromanipulacao_oocitos(conn, prontuario):
    """Query view_micromanipulacao_oocitos for a specific prontuario"""
    print(f"\n=== VIEW_MICROMANIPULACAO_OOCITOS ===")
    
    # First get codigo_ficha from view_micromanipulacao for this prontuario
    codigos_query = f"""
    SELECT codigo_ficha FROM silver.view_micromanipulacao 
    WHERE prontuario = {prontuario}
    """
    
    codigos_df = conn.execute(codigos_query).df()
    if not codigos_df.empty:
        codigo_list = codigos_df['codigo_ficha'].tolist()
        placeholders = ','.join(['?'] * len(codigo_list))
        
        oocitos_query = f"""
        SELECT * FROM silver.view_micromanipulacao_oocitos
        WHERE id_micromanipulacao IN ({placeholders})
        ORDER BY id_micromanipulacao, id
        """
        
        df = conn.execute(oocitos_query, codigo_list).df()
        print(f"Found {len(df)} records in view_micromanipulacao_oocitos")
    else:
        print("No codigo_ficha found for this prontuario in view_micromanipulacao.")
        df = pd.DataFrame()
    
    return df


In [4]:
def query_tratamentos(conn, prontuario):
    """Query view_tratamentos for a specific prontuario"""
    print(f"\n=== VIEW_TRATAMENTOS ===")
    query = f"""
    SELECT * FROM silver.view_tratamentos 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_tratamentos")
    return df

def query_congelamentos_embrioes(conn, prontuario):
    """Query view_congelamentos_embrioes for a specific prontuario"""
    print(f"\n=== VIEW_CONGELAMENTOS_EMBRIOES ===")
    query = f"""
    SELECT * FROM silver.view_congelamentos_embrioes 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_congelamentos_embrioes")
    return df

def query_descongelamentos_embrioes(conn, prontuario):
    """Query view_descongelamentos_embrioes for a specific prontuario"""
    print(f"\n=== VIEW_DESCONGELAMENTOS_EMBRIOES ===")
    query = f"""
    SELECT * FROM silver.view_descongelamentos_embrioes 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_descongelamentos_embrioes")
    return df

def query_embrioes_congelados(conn, prontuario):
    """Query view_embrioes_congelados for a specific prontuario"""
    print(f"\n=== VIEW_EMBRIOES_CONGELADOS ===")
    query = f"""
    SELECT * FROM silver.view_embrioes_congelados 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_embrioes_congelados")
    return df

def query_congelamentos_ovulos(conn, prontuario):
    """Query view_congelamentos_ovulos for a specific prontuario"""
    print(f"\n=== VIEW_CONGELAMENTOS_OVULOS ===")
    query = f"""
    SELECT * FROM silver.view_congelamentos_ovulos 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_congelamentos_ovulos")
    return df

def query_descongelamentos_ovulos(conn, prontuario):
    """Query view_descongelamentos_ovulos for a specific prontuario"""
    print(f"\n=== VIEW_DESCONGELAMENTOS_OVULOS ===")
    query = f"""
    SELECT * FROM silver.view_descongelamentos_ovulos 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_descongelamentos_ovulos")
    return df

def query_ovulos_congelados(conn, prontuario):
    """Query view_ovulos_congelados for a specific prontuario"""
    print(f"\n=== VIEW_OVULOS_CONGELADOS ===")
    query = f"""
    SELECT * FROM silver.view_ovulos_congelados 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_ovulos_congelados")
    return df


## Query Functions - Huntington Data Lake Tables


In [5]:
def query_clinisys_embrioes(conn, prontuario):
    """Query gold.clinisys_embrioes for a specific prontuario"""
    print(f"\n=== GOLD.CLINISYS_EMBRIOES ===")
    query = f"""
    SELECT * FROM gold.clinisys_embrioes 
    WHERE micro_prontuario = {prontuario}
    ORDER BY micro_Data_DL DESC, micro_codigo_ficha DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.clinisys_embrioes")
    return df

def query_embryoscope_embrioes(conn, prontuario):
    """Query gold.embryoscope_embrioes for a specific prontuario"""
    print(f"\n=== GOLD.EMBRYOSCOPE_EMBRIOES ===")
    query = f"""
    SELECT * FROM gold.embryoscope_embrioes 
    WHERE prontuario = {prontuario}
    ORDER BY embryo_KIDDate DESC, treatment_TreatmentName DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.embryoscope_embrioes")
    return df

def query_embryoscope_clinisys_combined(conn, prontuario):
    """Query gold.embryoscope_clinisys_combined for a specific prontuario"""
    print(f"\n=== GOLD.EMBRYOSCOPE_CLINISYS_COMBINED ===")
    query = f"""
    SELECT * FROM gold.embryoscope_clinisys_combined 
    WHERE micro_prontuario = {prontuario}
    ORDER BY micro_Data_DL DESC, micro_codigo_ficha DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.embryoscope_clinisys_combined")
    return df


In [6]:
def query_planilha_embriologia(conn, prontuario):
    """Query silver.planilha_embriologia for a specific prontuario"""
    print(f"\n=== SILVER.PLANILHA_EMBRIOLOGIA ===")
    query = f"""
    SELECT * FROM silver.planilha_embriologia 
    WHERE PIN = {prontuario}
    ORDER BY "DATA DA FET" DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in silver.planilha_embriologia")
    return df

def query_planilha_embryoscope_combined(conn, prontuario):
    """Query gold.planilha_embryoscope_combined for a specific prontuario"""
    print(f"\n=== GOLD.PLANILHA_EMBRYOSCOPE_COMBINED ===")
    query = f"""
    SELECT * FROM gold.planilha_embryoscope_combined 
    WHERE "micro_prontuario" = {prontuario} OR "planilha_PIN" = {prontuario}
    ORDER BY "descong_em_DataTransferencia" DESC, "planilha_DATA_DA_FET" DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.planilha_embryoscope_combined")
    return df


## Main Query Function


In [7]:
def query_embryo_data(prontuario):
    """
    Query all embryo-related data for a given prontuario/PIN.
    
    Args:
        prontuario (str or int): The patient prontuario number
        
    Returns:
        dict: Dictionary containing all DataFrames with results
    """
    print(f"\n{'='*80}")
    print(f"QUERYING EMBRYO DATA FOR PRONTUARIO: {prontuario}")
    print(f"Query executed at: {datetime.now()}")
    print(f"{'='*80}")
    
    # Connect to databases
    clinisys_con = get_clinisys_connection()
    huntington_con = get_huntington_connection()
    
    try:
        # Query clinisys tables
        df_micromanipulacao = query_micromanipulacao(clinisys_con, prontuario)
        df_oocitos = query_micromanipulacao_oocitos(clinisys_con, prontuario)
        df_tratamentos = query_tratamentos(clinisys_con, prontuario)
        df_congelamentos_embrioes = query_congelamentos_embrioes(clinisys_con, prontuario)
        df_descongelamentos_embrioes = query_descongelamentos_embrioes(clinisys_con, prontuario)
        df_embrioes_congelados = query_embrioes_congelados(clinisys_con, prontuario)
        df_congelamentos_ovulos = query_congelamentos_ovulos(clinisys_con, prontuario)
        df_descongelamentos_ovulos = query_descongelamentos_ovulos(clinisys_con, prontuario)
        df_ovulos_congelados = query_ovulos_congelados(clinisys_con, prontuario)
        
        # Query huntington data lake tables
        df_clinisys_embrioes = query_clinisys_embrioes(huntington_con, prontuario)
        df_embryoscope_embrioes = query_embryoscope_embrioes(huntington_con, prontuario)
        df_embryoscope_clinisys_combined = query_embryoscope_clinisys_combined(huntington_con, prontuario)
        # df_embryoscope_treatment = query_embryoscope_treatment(huntington_con, prontuario)
        df_planilha_embriologia = query_planilha_embriologia(huntington_con, prontuario)
        df_planilha_embryoscope_combined = query_planilha_embryoscope_combined(huntington_con, prontuario)
        
        # Summary
        print(f"\n{'='*80}")
        print(f"QUERY SUMMARY FOR PRONTUARIO {prontuario}")
        print(f"{'='*80}")
        print(f"- view_micromanipulacao: {len(df_micromanipulacao)} records")
        print(f"- view_micromanipulacao_oocitos: {len(df_oocitos)} records")
        print(f"- view_tratamentos: {len(df_tratamentos)} records")
        print(f"- view_congelamentos_embrioes: {len(df_congelamentos_embrioes)} records")
        print(f"- view_descongelamentos_embrioes: {len(df_descongelamentos_embrioes)} records")
        print(f"- view_embrioes_congelados: {len(df_embrioes_congelados)} records")
        print(f"- view_congelamentos_ovulos: {len(df_congelamentos_ovulos)} records")
        print(f"- view_descongelamentos_ovulos: {len(df_descongelamentos_ovulos)} records")
        print(f"- view_ovulos_congelados: {len(df_ovulos_congelados)} records")
        print(f"- gold.clinisys_embrioes: {len(df_clinisys_embrioes)} records")
        print(f"- gold.embryoscope_embrioes: {len(df_embryoscope_embrioes)} records")
        print(f"- gold.embryoscope_clinisys_combined: {len(df_embryoscope_clinisys_combined)} records")
        print(f"- silver.planilha_embriologia: {len(df_planilha_embriologia)} records")
        print(f"- gold.planilha_embryoscope_combined: {len(df_planilha_embryoscope_combined)} records")
        print(f"{'='*80}")
        
        # Return results as dictionary
        results = {
            'micromanipulacao': df_micromanipulacao,
            'micromanipulacao_oocitos': df_oocitos,
            'tratamentos': df_tratamentos,
            'congelamentos_embrioes': df_congelamentos_embrioes,
            'descongelamentos_embrioes': df_descongelamentos_embrioes,
            'embrioes_congelados': df_embrioes_congelados,
            'congelamentos_ovulos': df_congelamentos_ovulos,
            'descongelamentos_ovulos': df_descongelamentos_ovulos,
            'ovulos_congelados': df_ovulos_congelados,
            'clinisys_embrioes': df_clinisys_embrioes,
            'embryoscope_embrioes': df_embryoscope_embrioes,
            'embryoscope_clinisys_combined': df_embryoscope_clinisys_combined,
            'planilha_embriologia': df_planilha_embriologia,
            'planilha_embryoscope_combined': df_planilha_embryoscope_combined
        }
        
        return results
        
    finally:
        # Close connections
        clinisys_con.close()
        huntington_con.close()
        print("\nDatabase connections closed.")


## Example Usage


In [65]:
# Example usage
prontuario = 150547
# Query all embryo data
results = query_embryo_data(prontuario)



QUERYING EMBRYO DATA FOR PRONTUARIO: 150547
Query executed at: 2026-01-20 17:08:34.721813

=== VIEW_MICROMANIPULACAO ===
Found 1 records in view_micromanipulacao

=== VIEW_MICROMANIPULACAO_OOCITOS ===
Found 12 records in view_micromanipulacao_oocitos

=== VIEW_TRATAMENTOS ===
Found 1 records in view_tratamentos

=== VIEW_CONGELAMENTOS_EMBRIOES ===
Found 1 records in view_congelamentos_embrioes

=== VIEW_DESCONGELAMENTOS_EMBRIOES ===
Found 0 records in view_descongelamentos_embrioes

=== VIEW_EMBRIOES_CONGELADOS ===
Found 1 records in view_embrioes_congelados

=== VIEW_CONGELAMENTOS_OVULOS ===
Found 1 records in view_congelamentos_ovulos

=== VIEW_DESCONGELAMENTOS_OVULOS ===
Found 1 records in view_descongelamentos_ovulos

=== VIEW_OVULOS_CONGELADOS ===
Found 12 records in view_ovulos_congelados

=== GOLD.CLINISYS_EMBRIOES ===
Found 12 records in gold.clinisys_embrioes

=== GOLD.EMBRYOSCOPE_EMBRIOES ===
Found 11 records in gold.embryoscope_embrioes

=== GOLD.EMBRYOSCOPE_CLINISYS_COMBIN

## Display Results

### 1. Micromanipulacao


In [66]:
print("=== MICROMANIPULACAO DATA ===")
micromanipulacao_data = results['micromanipulacao']
print(f"Records: {len(micromanipulacao_data)}")
if not micromanipulacao_data.empty:
    display(micromanipulacao_data)
else:
    print("No micromanipulacao data found.")


=== MICROMANIPULACAO DATA ===
Records: 1


,codigo_ficha,numero_caso,prontuario,IdadeEsposa_DG,IdadeMarido_DG,Data_DL,horario_inicial_fert,horario_final_fert,Aspiracao_DL,medico_id,responsavel_puncao_id,recepcao_ovulos,Denud_DL,checagem_fertilizacao,checagem_d3,checagem_d5,checagem_d6,checagem_d7,foliculos,oocitos,ICSIDescongelados,pgd_oocito,pgd_d5,data_procedimento,tecnica_injetados,motilidade,morfologia_injetados,gradiente_descontinuo,unidade_id,responsavel_denud_id,profissional_fertilizacao_id,profissional_d3_id,profissional_d5_id,profissional_d6_id,profissional_d7_id,materiais_insumos,local_coleta,viscosidade,volume_amostra_utilizada,morfologia_pre,morfologia_pos,zymot,observacoes_resumo_ciclos,agulha_icsi,lote_agulha_icsi,validade_agulha_icsi,agulha_aspir,lote_agulha_aspir,validade_agulha_aspir,incubadora,abstinencia_dias,meios,responsavel_oocitos_id,fonte,inicio_ps,termino_ps,volume_ps1,CongelamentoOvulos,concentracao_progressivos_pre,conc_ps1,EstadoSptz,check_gradiente_ps,check_swimup,volume_ps2,concentracao_progressivos_pos,conc_ps2,comentarios_ps,responsavel_ps,timelapse,transferidos,Transfer_D5,medico_responsavel_id,azoospermia,Observacoes,responsavel_labfiv,responsavel_labfiv_data,tipo_paciente,alteracoes_oocitarias_normal,alteracoes_oocitarias_granulacao,alteracoes_oocitarias_pelucida,alteracoes_oocitarias_oval,alteracoes_oocitarias_inclusoes,alteracoes_oocitarias_pervitelineo_aumentado,alteracoes_oocitarias_irregular,alteracoes_oocitarias_necrotica,alteracoes_oocitarias_pervitelineo_granular,alteracoes_oocitarias_ser,alteracoes_oocitarias_corpusculo_frag,alteracoes_oocitarias_vacuolos,alteracoes_oocitarias_corpusculo_aum,alteracoes_oocitarias_membrana,alteracoes_oocitarias_corpusculo_peq,alteracoes_oocitarias_citoplasma,alteracoes_oocitarias_corpusculo_deg,controle_anual,maia,hash,extraction_timestamp
0,28918,4575/25,150547,42,46,2025-12-13,13:20,13:47,None,31,None,Não,None,08:00,08:00,08:00,08:00,None,None,None,Sim,None,None,2025-12-13,ICSI,11 espermatozoides móveis progressivos\r\n,11 espermatozoides normais\r\n,1,0000000001,None,3492,3505,3492,3492,None,EMBRYOGLUE - LOTE: 052500 - 28/01/2026\t\t\r\nCSCM NX: LOTE: 50899 - 23/01/26\t\t\r\nDESVITRI INGÁ - LOTE: 25090614 - 03/03/26\t\t\r\nG-IVF PLUS - LOTE: 513577 - 13/01/26\t\t\r\nG-MOPS PLUS - LOTE: 513653 - 09/01/2026\t\t\r\nHYALURONIDASE - LOTE: FP25HY22 - 07/2026\t\t\r\nISOLATE - LOTE: 39167 - 31/10/2026\t\t\r\nGV GRADIENTE - LOTE: 250603681 - 23/03/2026\t\t\r\nOVOIL - LOTE: 513178 - 26/02/2026\t\t\r\nOLEO HEAVY - LOTE: F0503 - 30/11/26\t\t\r\nPBS - LOTE: 25090689 - 30/09/26\t\t\r\nPVP - LOTE: 43957 - 30/01/26\t\t\r\nSORO INGÁ - LOTE: 25080568 - 02/02/2026\t\t\r\nVITRI INGÁ - LOTE: 25080540 -04/02/26\t\t\r\nGV HEPES - LOTE: 25100738 - 20/04/26\t\t\r\nKITAZATO THAWING MEDIA - LOTE: O0522 - 16/05/26\t\t\r\nKITAZATO VITRIFICATION - LOTE: O0531 - 16/15/26\t\t\r\nAGULHA BIÓPSIA - LOTE:619030559 - 27/08/28\t\t\r\nAGULHA COLETA - LOTE: 619034728 - 20/11/28\t\t\r\nAGULHA HOLDING - LOTE: N214258 - 29/02/28\t\t\r\nAGULHA ICSI - LOTE: N215626 - 31/03/2028\t\t\r\nCATÉTER OBTURADOR - LOTE: 16457692 - 10/02/2028\t\t\r\nCATÉTER SOFT ECO WALLACE - LOTE:619044981 - 29/05/2027\t\t\r\nCATÉTER SYDNEI - LOTE: 16511401 - 16/03/2028\t\t\r\nMICROPIPETA DE DENUDAÇÃO - LOTE: S12072CA - 25/12/27\t\t\r\nPIPETA PASTEUR VIDRO - LOTE: 550018190 - 12/11/27\t\t\r\nPIPETA SOROLÓGICA 10ML - LOTE: 5075073 - 27/02/2030\t\t\r\nSTRIPPER 135 - LOTE- 442502/19/02 - 04/28\t\t\r\nSTRIPPER 175 - LOTE: 442502/18/02 - 03/28\t\t\r\nSTRIPPER 200 - LOTE: 442502/19/03 - 03/2028\t\t\r\nSTRIPPER 275 - LOTE: 442503/30/03 - 04/2028\t\t\r\nCRIOTUBO 2ML - LOTE: 14824056 - 30/05/2029\t\t\r\nCRIOTUBO 5ML - LOTE: 23724001 - VAL: 29/08/2029\t\t\r\nCRYOTOP - LOTE: S50422 - 22/04/28\t\t\r\nTUBO 14ML - LOTE: 13825011 - VAL: 23/10/2029\t\t\r\n\t\t\r\nPLACA 15X35 - LOTE: 24070437 - 10/07/2029\t\t\r\nPLACA 15x60 - LOTE: 24060383 - VAL: 07/06/2029\t\t\r\nPOÇO CENTRAL - LOTE: 24080494 - VAL: 08/08/2029\t\t\r\nPLACA ICSI - LOTE: 230211 - 13/02/2026\t\t\r\nEMBRYOSCOPE -

### 2. Micromanipulacao Oocitos


In [67]:
print("=== MICROMANIPULACAO OOCITOS DATA ===")
oocitos_data = results['micromanipulacao_oocitos']
print(f"Records: {len(oocitos_data)}")
if not oocitos_data.empty:
    display(oocitos_data)
else:
    print("No micromanipulacao oocitos data found.")


=== MICROMANIPULACAO OOCITOS DATA ===
Records: 12


,id,id_micromanipulacao,diaseguinte,Maturidade,Embriologista,PI,TCD,PGD,ResultadoPGD,IdentificacaoPGD,Fertilizacao,CorpusculoPolar,ComentariosDepois,OocitoDoado,OrigemOocito,InseminacaoOocito,NClivou_D3,NCelulas_D3,Frag_D3,Blastomero_D3,NClivou_D4,NCelulas_D4,Compactando_D4,MassaInterna_D4,Trofoblasto_D4,NClivou_D5,NCelulas_D5,Compactando_D5,MassaInterna_D5,Trofoblasto_D5,NClivou_D6,NCelulas_D6,Compactando_D6,MassaInterna_D6,Trofoblasto_D6,NClivou_D7,NCelulas_D7,Compactando_D7,MassaInterna_D7,Trofoblasto_D7,score_maia,relatorio_ia,hash,extraction_timestamp,flag_embryoscope,embryo_number
0,292126,28918,Não,MII,3888,None,Descartado,None,None,None,2PN,None,Fresco,Não,Descongelado,ICSI,Sim,3,3,Simétricos,None,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1a85e0def39ef9a614bfa86ebf56dcee,2026-01-16 19:39:10,1,1
1,292127,28918,Não,MII,3888,None,Descartado,None,None,None,1CP,None,Fresco,Não,Descongelado,ICSI,Sim,2,3,Simétricos,None,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1dd6ebd1dce1ebfe54cab3b045261e80,2026-01-16 19:39:10,1,2
2,292128,28918,Não,MII,3888,None,Criopreservado,Não,None,None,2PN,None,Fresco,Não,Descongelado,ICSI,Sim,7,2,Simétricos,None,None,None,None,None,Sim,None,Blastocisto Grau 5,B,B,None,None,None,None,None,None,None,None,None,None,2.39,Sim,f8f66c7570defe7c9817ce206d38a40c,2026-01-16 19:39:10,1,3
3,292129,28918,Não,MII,3888,None,Descartado,None,None,None,2PN,None,Fresco,Não,Descongelado,ICSI,Sim,2,3,Simétricos,None,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0949f4f432279e5306138af50438243e,2026-01-16 19:39:10,1,4
4,292130,28918,Não,MII,3888,None,Descartado,None,None,None,2PN,None,Fresco,Não,Descongelado,ICSI,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,6929cec379f42cdc4e165d2c11deb0b5,2026-01-16 19:39:10,1,5
5,292131,28918,Não,MII,3888,None,Descartado,None,None,None,2PN,None,Fresco,Não,Descongelado,ICSI,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,527c2dc25213ba0328d750aba4819e69,2026-01-16 19:39:10,1,6
6,292132,28918,Não,MII,3888,None,Descartado,None,None,None,2PN,None,Fresco,Não,Descongelado,ICSI,Sim,7,2,Simétricos,None,None,None,None,None,Sim,None,Blastocisto Grau 1,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,fad0bb72600112a006c7984de5b663ec,2026-01-16 19:39:10,1,7
7,292133,28918,Não,MII,3888,None,Descartado,None,None,None,2PN,None,Fresco,Não,Descongelado,ICSI,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,72afe00080ddec16dc1bef2ec17cd3f3,2026-01-16 19:39:10,1,8
8,292134,28918,Não,MII,3888,None,Transferido,Não,None,None,2PN,None,Fresco,Não,Descongelado,ICSI,Sim,9,2,Simétricos,None,None,None,None,None,Sim,None,Blastocisto Grau 4,B,B,None,None,None,None,None,None,None,None,None,None,6.49,Sim,49a3eb7a63a0adce6122344a4e908965,2026-01-16 19:39:10,1,9
9,292135,28918,Não,MII,3888,None,Descartado,None,None,None,2PN,None,Fresco,Não,Descongelado,ICSI,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,cd8b93f0e97bcf676a6af9d83552bc28,2026-01-16 19:39:10,1,10


### 3. Tratamentos


In [68]:
print("=== TRATAMENTOS DATA ===")
tratamentos_data = results['tratamentos']
print(f"Records: {len(tratamentos_data)}")
if not tratamentos_data.empty:
    display(tratamentos_data)
else:
    print("No tratamentos data found.")


=== TRATAMENTOS DATA ===
Records: 1


,id,prontuario,unidade,idade_esposa,idade_marido,paciente_tratamento,tentativa,data_procedimento,tipo_procedimento,tipo_inducao,protocolo,tipo_preparo_endometrial,origem_ovulo,status_ovulo,prontuario_doadora,origem_espermatozoide,fonte_espermatozoide,status_espermatozoide,origem_material,utero_substituicao,prontuario_genitores,doacao_ovulos,doacao_embrioes,pgt,era_auxiliar,emma_resultado,alice_resultado,picsi,hatching_od,hatching_oe,medicacao_parental,peso_paciente,altura_paciente,fumante_paciente,data_dum,peso_conjuge,altura_conjuge,fumante_conjuge,fator_infertilidade1,fator_infertilidade2,foliculos_antrais,data_inicio_inducao,dias_inducao_preparo,paciente_inducao,data_inicio_agonista,utilizado_hcg,data_hcg,horario_planejado_hcg,gonapeptyl,dose_gonapeptyl,data_inicio_progesterona,progesterona_vaginal,data_congelamento,metodo_descongelamento,valor_palheta,manter_excedentes,dia_transferencia,data_transferencia,medida_endometrio,padrao_endometrio,motivo_nao_transferir,num_sacos_gestacionais,resultado_tratamento,analise_genetica,altura_bebe1,via_parto_bebe1,altura_bebe2,via_parto_bebe2,altura_bebe3,via_parto_bebe3,altura_bebe4,via_parto_bebe4,observacoes_gerais,usuario_responsavel,responsavel_informacoes,hash,extraction_timestamp
0,40152,150547,1,42,46,esposa,1,2025-12-13,Ciclo a Fresco FIV,Ciclo Natural,None,Hormonal,Homólogo,Criopreservado,NaN,Homólogo,Sêmen,Fresco,None,Não,NaN,Não,Não,Não,None,None,None,Não,None,Sim,None,62.0,1.59,None,2025-12-01,NaN,NaN,None,Insuficiência ovariana,None,None,2025-12-01,18,esposa,None,Não utilizado,None,None,None,None,13/12/2025,1,NaT,None,None,None,5,2025-12-18,"7,8",Trilaminar,None,1,Gestação Clínica,Não,None,None,None,None,None,None,None,None,None,3508,31,59db346811db94e0c554ad828e9fee27,2026-01-16 19:35:37


### 4. Congelamentos Embrioes


In [69]:
print("=== CONGELAMENTOS EMBRIOES DATA ===")
congelamentos_data = results['congelamentos_embrioes']
print(f"Records: {len(congelamentos_data)}")
if not congelamentos_data.empty:
    display(congelamentos_data)
else:
    print("No congelamentos embrioes data found.")


=== CONGELAMENTOS EMBRIOES DATA ===
Records: 1


,id,CodCongelamento,Unidade,prontuario,paciente,Data,Hora,Ciclo,CicloRecongelamento,NEmbrioes,NPailletes,Identificacao,Tambor,Cane,Cane2,Tecnica,D5,D6,Observacoes,BiologoFIV,BiologoFIV2,status_financeiro,hash,extraction_timestamp
0,31177,E3487/25,1,150547,esposa,2025-12-18,1900-01-01 18:00:00,4575/25,Não,1,1,E3 BRANCO R23,None,None,None,Vitrificação,1.0,NaN,None,Laura Belomo,Otavio Santos,Cobrar,f12f9465246f77ed359ce404229fa91e,2026-01-16 19:35:47


### 5. Descongelamentos Embrioes


In [70]:
print("=== DESCONGELAMENTOS EMBRIOES DATA ===")
descongelamentos_data = results['descongelamentos_embrioes']
print(f"Records: {len(descongelamentos_data)}")
if not descongelamentos_data.empty:
    display(descongelamentos_data)
else:
    print("No descongelamentos embrioes data found.")


=== DESCONGELAMENTOS EMBRIOES DATA ===
Records: 0
No descongelamentos embrioes data found.


### 6. Embrioes Congelados


In [71]:
print("=== EMBRIOES CONGELADOS DATA ===")
embrioes_congelados_data = results['embrioes_congelados']
print(f"Records: {len(embrioes_congelados_data)}")
if not embrioes_congelados_data.empty:
    display(embrioes_congelados_data)
    # display(embrioes_congelados_data[embrioes_congelados_data['id_descongelamento']!=0])
else:
    print("No embrioes congelados data found.")


=== EMBRIOES CONGELADOS DATA ===
Records: 1


,id,id_oocito,id_congelamento,id_descongelamento,prontuario,pailletes,pailletes_id,embriao,qualidade,comentarios,comentarios_descongelamento,ah,dia5_2,dia5_3,dia5_4,transferidos,pgd,colapso,pgd_congelamento,resultado_pgd_congelamento,mito_teste_pgd_congelamento,dia_congelamento,score_maia,tanque_amostra,caneca_amostra,rack_amostra,hash,extraction_timestamp
0,93818,292128,31177,0,150547,P3,E3 BRANCO R23,(3),Blastocisto Grau 5 - B - B,None,None,None,None,None,None,None,None,NÃO BIOPSIADO,Não,None,6.4,D5,2.39,E3,Branca,R23,d66277c2a95cd378ade784da96166735,2026-01-16 19:36:05


### 7. Clinisys Embrioes


In [72]:
print("=== CLINISYS EMBRIOES DATA ===")
clinisys_embrioes_data = results['clinisys_embrioes']
print(f"Records: {len(clinisys_embrioes_data)}")
if not clinisys_embrioes_data.empty:
    display(clinisys_embrioes_data.head(10))
else:
    print("No clinisys embrioes data found.")


=== CLINISYS EMBRIOES DATA ===
Records: 12


,oocito_id,oocito_id_micromanipulacao,oocito_diaseguinte,oocito_Maturidade,oocito_Embriologista,oocito_PI,oocito_TCD,oocito_PGD,oocito_ResultadoPGD,oocito_IdentificacaoPGD,oocito_Fertilizacao,oocito_CorpusculoPolar,oocito_ComentariosDepois,oocito_OocitoDoado,oocito_OrigemOocito,oocito_InseminacaoOocito,oocito_NClivou_D3,oocito_NCelulas_D3,oocito_Frag_D3,oocito_Blastomero_D3,oocito_NClivou_D4,oocito_NCelulas_D4,oocito_Compactando_D4,oocito_MassaInterna_D4,oocito_Trofoblasto_D4,oocito_NClivou_D5,oocito_NCelulas_D5,oocito_Compactando_D5,oocito_MassaInterna_D5,oocito_Trofoblasto_D5,oocito_NClivou_D6,oocito_NCelulas_D6,oocito_Compactando_D6,oocito_MassaInterna_D6,oocito_Trofoblasto_D6,oocito_NClivou_D7,oocito_NCelulas_D7,oocito_Compactando_D7,oocito_MassaInterna_D7,oocito_Trofoblasto_D7,oocito_score_maia,oocito_relatorio_ia,oocito_flag_embryoscope,oocito_embryo_number,micro_codigo_ficha,micro_numero_caso,micro_prontuario,micro_IdadeEsposa_DG,micro_IdadeMarido_DG,micro_Data_DL,micro_horario_inicial_fert,micro_horario_final_fert,micro_Aspiracao_DL,micro_medico_id,micro_responsavel_puncao_id,micro_recepcao_ovulos,micro_Denud_DL,micro_checagem_fertilizacao,micro_checagem_d3,micro_checagem_d5,micro_checagem_d6,micro_checagem_d7,micro_foliculos,micro_oocitos,micro_ICSIDescongelados,micro_pgd_oocito,micro_pgd_d5,micro_data_procedimento,micro_tecnica_injetados,micro_motilidade,micro_morfologia_injetados,micro_gradiente_descontinuo,micro_unidade_id,micro_responsavel_denud_id,micro_profissional_fertilizacao_id,micro_profissional_d3_id,micro_profissional_d5_id,micro_profissional_d6_id,micro_profissional_d7_id,micro_materiais_insumos,micro_local_coleta,micro_viscosidade,micro_volume_amostra_utilizada,micro_morfologia_pre,micro_morfologia_pos,micro_zymot,micro_observacoes_resumo_ciclos,micro_agulha_icsi,micro_lote_agulha_icsi,micro_validade_agulha_icsi,micro_agulha_aspir,micro_lote_agulha_aspir,micro_validade_agulha_aspir,micro_incubadora,micro_abstinencia_dias,micro_meios,micro_responsavel_oocitos_id,micro_fonte,micro_inicio_ps,micro_termino_ps,micro_volume_ps1,micro_CongelamentoOvulos,micro_concentracao_progressivos_pre,micro_conc_ps1,micro_EstadoSptz,micro_check_gradiente_ps,micro_check_swimup,micro_volume_ps2,micro_concentracao_progressivos_pos,micro_conc_ps2,micro_comentarios_ps,micro_responsavel_ps,micro_timelapse,micro_transferidos,micro_Transfer_D5,micro_medico_responsavel_id,micro_azoospermia,micro_Observacoes,micro_responsavel_labfiv,micro_responsavel_labfiv_data,micro_tipo_paciente,micro_alteracoes_oocitarias_normal,micro_alteracoes_oocitarias_granulacao,micro_alteracoes_oocitarias_pelucida,micro_alteracoes_oocitarias_oval,micro_alteracoes_oocitarias_inclusoes,micro_alteracoes_oocitarias_pervitelineo_aumentado,micro_alteracoes_oocitarias_irregular,micro_alteracoes_oocitarias_necrotica,micro_alteracoes_oocitarias_pervitelineo_granular,micro_alteracoes_oocitarias_ser,micro_alteracoes_oocitarias_corpusculo_frag,micro_alteracoes_oocitarias_vacuolos,micro_alteracoes_oocitarias_corpusculo_aum,micro_alteracoes_oocitarias_membrana,micro_alteracoes_oocitarias_corpusculo_peq,micro_alteracoes_oocitarias_citoplasma,micro_alteracoes_oocitarias_corpusculo_deg,micro_controle_anual,micro_maia,cong_em_id,cong_em_CodCongelamento,cong_em_Unidade,cong_em_prontuario,cong_em_paciente,cong_em_Data,cong_em_Hora,cong_em_Ciclo,cong_em_CicloRecongelamento,cong_em_NEmbrioes,cong_em_NPailletes,cong_em_Identificacao,cong_em_Tambor,cong_em_Cane,cong_em_Cane2,cong_em_Tecnica,cong_em_D5,cong_em_D6,cong_em_Observacoes,cong_em_BiologoFIV,cong_em_BiologoFIV2,cong_em_status_financeiro,descong_em_id,descong_em_CodDescongelamento,descong_em_Unidade,descong_em_prontuario,descong_em_doadora,descong_em_DataCongelamento,descong_em_DataDescongelamento,descong_em_Ciclo,descong_em_Identificacao,descong_em_CodCongelamento,descong_em_Tambor,descong_em_Cane,descong_em_PailletesDescongeladas,descong_em_Tecnica,descong_em_DataTransferencia,descong_em_Incubadora,descong_em_transferido

In [73]:
clinisys_embrioes_data[clinisys_embrioes_data['oocito_id'].isin([184181])]

,oocito_id,oocito_id_micromanipulacao,oocito_diaseguinte,oocito_Maturidade,oocito_Embriologista,oocito_PI,oocito_TCD,oocito_PGD,oocito_ResultadoPGD,oocito_IdentificacaoPGD,oocito_Fertilizacao,oocito_CorpusculoPolar,oocito_ComentariosDepois,oocito_OocitoDoado,oocito_OrigemOocito,oocito_InseminacaoOocito,oocito_NClivou_D3,oocito_NCelulas_D3,oocito_Frag_D3,oocito_Blastomero_D3,oocito_NClivou_D4,oocito_NCelulas_D4,oocito_Compactando_D4,oocito_MassaInterna_D4,oocito_Trofoblasto_D4,oocito_NClivou_D5,oocito_NCelulas_D5,oocito_Compactando_D5,oocito_MassaInterna_D5,oocito_Trofoblasto_D5,oocito_NClivou_D6,oocito_NCelulas_D6,oocito_Compactando_D6,oocito_MassaInterna_D6,oocito_Trofoblasto_D6,oocito_NClivou_D7,oocito_NCelulas_D7,oocito_Compactando_D7,oocito_MassaInterna_D7,oocito_Trofoblasto_D7,oocito_score_maia,oocito_relatorio_ia,oocito_flag_embryoscope,oocito_embryo_number,micro_codigo_ficha,micro_numero_caso,micro_prontuario,micro_IdadeEsposa_DG,micro_IdadeMarido_DG,micro_Data_DL,micro_horario_inicial_fert,micro_horario_final_fert,micro_Aspiracao_DL,micro_medico_id,micro_responsavel_puncao_id,micro_recepcao_ovulos,micro_Denud_DL,micro_checagem_fertilizacao,micro_checagem_d3,micro_checagem_d5,micro_checagem_d6,micro_checagem_d7,micro_foliculos,micro_oocitos,micro_ICSIDescongelados,micro_pgd_oocito,micro_pgd_d5,micro_data_procedimento,micro_tecnica_injetados,micro_motilidade,micro_morfologia_injetados,micro_gradiente_descontinuo,micro_unidade_id,micro_responsavel_denud_id,micro_profissional_fertilizacao_id,micro_profissional_d3_id,micro_profissional_d5_id,micro_profissional_d6_id,micro_profissional_d7_id,micro_materiais_insumos,micro_local_coleta,micro_viscosidade,micro_volume_amostra_utilizada,micro_morfologia_pre,micro_morfologia_pos,micro_zymot,micro_observacoes_resumo_ciclos,micro_agulha_icsi,micro_lote_agulha_icsi,micro_validade_agulha_icsi,micro_agulha_aspir,micro_lote_agulha_aspir,micro_validade_agulha_aspir,micro_incubadora,micro_abstinencia_dias,micro_meios,micro_responsavel_oocitos_id,micro_fonte,micro_inicio_ps,micro_termino_ps,micro_volume_ps1,micro_CongelamentoOvulos,micro_concentracao_progressivos_pre,micro_conc_ps1,micro_EstadoSptz,micro_check_gradiente_ps,micro_check_swimup,micro_volume_ps2,micro_concentracao_progressivos_pos,micro_conc_ps2,micro_comentarios_ps,micro_responsavel_ps,micro_timelapse,micro_transferidos,micro_Transfer_D5,micro_medico_responsavel_id,micro_azoospermia,micro_Observacoes,micro_responsavel_labfiv,micro_responsavel_labfiv_data,micro_tipo_paciente,micro_alteracoes_oocitarias_normal,micro_alteracoes_oocitarias_granulacao,micro_alteracoes_oocitarias_pelucida,micro_alteracoes_oocitarias_oval,micro_alteracoes_oocitarias_inclusoes,micro_alteracoes_oocitarias_pervitelineo_aumentado,micro_alteracoes_oocitarias_irregular,micro_alteracoes_oocitarias_necrotica,micro_alteracoes_oocitarias_pervitelineo_granular,micro_alteracoes_oocitarias_ser,micro_alteracoes_oocitarias_corpusculo_frag,micro_alteracoes_oocitarias_vacuolos,micro_alteracoes_oocitarias_corpusculo_aum,micro_alteracoes_oocitarias_membrana,micro_alteracoes_oocitarias_corpusculo_peq,micro_alteracoes_oocitarias_citoplasma,micro_alteracoes_oocitarias_corpusculo_deg,micro_controle_anual,micro_maia,cong_em_id,cong_em_CodCongelamento,cong_em_Unidade,cong_em_prontuario,cong_em_paciente,cong_em_Data,cong_em_Hora,cong_em_Ciclo,cong_em_CicloRecongelamento,cong_em_NEmbrioes,cong_em_NPailletes,cong_em_Identificacao,cong_em_Tambor,cong_em_Cane,cong_em_Cane2,cong_em_Tecnica,cong_em_D5,cong_em_D6,cong_em_Observacoes,cong_em_BiologoFIV,cong_em_BiologoFIV2,cong_em_status_financeiro,descong_em_id,descong_em_CodDescongelamento,descong_em_Unidade,descong_em_prontuario,descong_em_doadora,descong_em_DataCongelamento,descong_em_DataDescongelamento,descong_em_Ciclo,descong_em_Identificacao,descong_em_CodCongelamento,descong_em_Tambor,descong_em_Cane,descong_em_PailletesDescongeladas,descong_em_Tecnica,descong_em_DataTransferencia,descong_em_Incubadora,descong_em_transferido

In [74]:
colunas = [
    'oocito_id','oocito_id_micromanipulacao', 'oocito_embryo_number', 'oocito_TCD', 
    'micro_codigo_ficha', 'micro_numero_caso', 'micro_Data_DL', 'micro_data_procedimento', 
    'cong_em_id', 'cong_em_CodCongelamento', 'cong_em_Ciclo', 'cong_em_NEmbrioes', 
    'emb_cong_id', 'emb_cong_id_oocito', 'emb_cong_id_congelamento', 'emb_cong_id_descongelamento','emb_cong_embriao', 
    'descong_em_id', 'descong_em_transferidos_transferencia'	, 'descong_em_CodDescongelamento', 'descong_em_Ciclo' , 'descong_em_DataDescongelamento', 'descong_em_DataTransferencia', 
    'trat_id', 'trat_data_procedimento', 'trat_data_transferencia', 'trat_resultado_tratamento'
]

clinisys_embrioes_data.loc[clinisys_embrioes_data['oocito_embryo_number'].isin([33,35]), colunas]

,oocito_id,oocito_id_micromanipulacao,oocito_embryo_number,oocito_TCD,micro_codigo_ficha,micro_numero_caso,micro_Data_DL,micro_data_procedimento,cong_em_id,cong_em_CodCongelamento,cong_em_Ciclo,cong_em_NEmbrioes,emb_cong_id,emb_cong_id_oocito,emb_cong_id_congelamento,emb_cong_id_descongelamento,emb_cong_embriao,descong_em_id,descong_em_transferidos_transferencia,descong_em_CodDescongelamento,descong_em_Ciclo,descong_em_DataDescongelamento,descong_em_DataTransferencia,trat_id,trat_data_procedimento,trat_data_transferencia,trat_resultado_tratamento


### 8. Embryoscope Embrioes


In [75]:
print("=== EMBRYOSCOPE EMBRIOES DATA ===")
embryoscope_embrioes_data = results['embryoscope_embrioes']
print(f"Records: {len(embryoscope_embrioes_data)}")
if not embryoscope_embrioes_data.empty:
    display(embryoscope_embrioes_data)
else:
    print("No embryoscope embrioes data found.")


=== EMBRYOSCOPE EMBRIOES DATA ===
Records: 11


,patient_PatientIDx,patient_PatientID,patient_FirstName,patient_LastName,patient_DateOfBirth,prontuario,patient_unit_huntington,treatment_PatientIDx,treatment_TreatmentName,treatment_unit_huntington,embryo_EmbryoID,embryo_KIDDate,embryo_KIDScore,embryo_KIDUser,embryo_KIDVersion,embryo_Description,embryo_EmbryoDescriptionID,embryo_EmbryoFate,embryo_FertilizationMethod,embryo_FertilizationTime,embryo_InstrumentNumber,embryo_Position,embryo_WellNumber,embryo_embryo_number,embryo_unit_huntington,embryo_Name_BlastExpandLast,embryo_Time_BlastExpandLast,embryo_Timestamp_BlastExpandLast,embryo_Value_BlastExpandLast,embryo_Name_BlastomereSize,embryo_Time_BlastomereSize,embryo_Timestamp_BlastomereSize,embryo_Value_BlastomereSize,embryo_Name_EVEN2,embryo_Time_EVEN2,embryo_Timestamp_EVEN2,embryo_Value_EVEN2,embryo_Name_EVEN4,embryo_Time_EVEN4,embryo_Timestamp_EVEN4,embryo_Value_EVEN4,embryo_Name_EVEN8,embryo_Time_EVEN8,embryo_Timestamp_EVEN8,embryo_Value_EVEN8,embryo_Name_FRAG2,embryo_Time_FRAG2,embryo_Timestamp_FRAG2,embryo_Value_FRAG2,embryo_Name_FRAG2CAT,embryo_Time_FRAG2CAT,embryo_Timestamp_FRAG2CAT,embryo_Value_FRAG2CAT,embryo_Name_FRAG4,embryo_Time_FRAG4,embryo_Timestamp_FRAG4,embryo_Value_FRAG4,embryo_Name_FRAG8,embryo_Time_FRAG8,embryo_Timestamp_FRAG8,embryo_Value_FRAG8,embryo_Name_Fragmentation,embryo_Time_Fragmentation,embryo_Timestamp_Fragmentation,embryo_Value_Fragmentation,embryo_Name_ICM,embryo_Time_ICM,embryo_Timestamp_ICM,embryo_Value_ICM,embryo_Name_Line,embryo_Time_Line,embryo_Timestamp_Line,embryo_Value_Line,embryo_Name_MN2Type,embryo_Time_MN2Type,embryo_Timestamp_MN2Type,embryo_Value_MN2Type,embryo_Name_MorphologicalGrade,embryo_Time_MorphologicalGrade,embryo_Timestamp_MorphologicalGrade,embryo_Value_MorphologicalGrade,embryo_Name_MultiNucleation,embryo_Time_MultiNucleation,embryo_Timestamp_MultiNucleation,embryo_Value_MultiNucleation,embryo_Name_Nuclei2,embryo_Time_Nuclei2,embryo_Timestamp_Nuclei2,embryo_Value_Nuclei2,embryo_Name_PN,embryo_Time_PN,embryo_Timestamp_PN,embryo_Value_PN,embryo_Name_Pulsing,embryo_Time_Pulsing,embryo_Timestamp_Pulsing,embryo_Value_Pulsing,embryo_Name_ReexpansionCount,embryo_Time_ReexpansionCount,embryo_Timestamp_ReexpansionCount,embryo_Value_ReexpansionCount,embryo_Name_Strings,embryo_Time_Strings,embryo_Timestamp_Strings,embryo_Value_Strings,embryo_Name_TE,embryo_Time_TE,embryo_Timestamp_TE,embryo_Value_TE,embryo_Name_t2,embryo_Time_t2,embryo_Timestamp_t2,embryo_Value_t2,embryo_Name_t3,embryo_Time_t3,embryo_Timestamp_t3,embryo_Value_t3,embryo_Name_t4,embryo_Time_t4,embryo_Timestamp_t4,embryo_Value_t4,embryo_Name_t5,embryo_Time_t5,embryo_Timestamp_t5,embryo_Value_t5,embryo_Name_t6,embryo_Time_t6,embryo_Timestamp_t6,embryo_Value_t6,embryo_Name_t7,embryo_Time_t7,embryo_Timestamp_t7,embryo_Value_t7,embryo_Name_t8,embryo_Time_t8,embryo_Timestamp_t8,embryo_Value_t8,embryo_Name_t9,embryo_Time_t9,embryo_Timestamp_t9,embryo_Value_t9,embryo_Name_tB,embryo_Time_tB,embryo_Timestamp_tB,embryo_Value_tB,embryo_Name_tEB,embryo_Time_tEB,embryo_Timestamp_tEB,embryo_Value_tEB,embryo_Name_tHB,embryo_Time_tHB,embryo_Timestamp_tHB,embryo_Value_tHB,embryo_Name_tM,embryo_Time_tM,embryo_Timestamp_tM,embryo_Value_tM,embryo_Name_tPB2,embryo_Time_tPB2,embryo_Timestamp_tPB2,embryo_Value_tPB2,embryo_Name_tPNa,embryo_Time_tPNa,embryo_Timestamp_tPNa,embryo_Value_tPNa,embryo_Name_tPNf,embryo_Time_tPNf,embryo_Timestamp_tPNf,embryo_Value_tPNf,embryo_Name_tSB,embryo_Time_tSB,embryo_Timestamp_tSB,embryo_Value_tSB,embryo_Name_tSC,embryo_Time_tSC,embryo_Timestamp_tSC,embryo_Value_tSC,embryo_PatientIDx,embryo_TreatmentName
0,PC1P7BHG_46004.4764412153,150547,"JAIME, THAIS J.",10/04/1983,1983-04-10,150547,Ibirapuera,PC1P7BHG_46004.4764412153,2025-2849,Ibirapuera,D2025.12.13_S04488_I3027_P-4,2025-12-14,NA,ADMIN,KIDScoreD5 v3.3,None,AA4,Unknown,None,2025-12-13 13:40:00,3027,6,4,4,Ibirapuera,None,NaN,None,None,None,NaN,None,None,None,NaN,None,None,None,NaN,None,None,None,NaN,None,None,None,NaN,None,None,None,NaN,None,None,None,NaN,N

In [76]:
embryoscope_embrioes_data.loc[embryoscope_embrioes_data['embryo_FertilizationTime'].isin(['2024-01-15 12:45:00'])].sort_values(by='embryo_embryo_number')

C:\Users\FilipeFurlanBellotti\AppData\Local\Temp\ipykernel_28664\2666312698.py:1: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  embryoscope_embrioes_data.loc[embryoscope_embrioes_data['embryo_FertilizationTime'].isin(['2024-01-15 12:45:00'])].sort_values(by='embryo_embryo_number')


,patient_PatientIDx,patient_PatientID,patient_FirstName,patient_LastName,patient_DateOfBirth,prontuario,patient_unit_huntington,treatment_PatientIDx,treatment_TreatmentName,treatment_unit_huntington,embryo_EmbryoID,embryo_KIDDate,embryo_KIDScore,embryo_KIDUser,embryo_KIDVersion,embryo_Description,embryo_EmbryoDescriptionID,embryo_EmbryoFate,embryo_FertilizationMethod,embryo_FertilizationTime,embryo_InstrumentNumber,embryo_Position,embryo_WellNumber,embryo_embryo_number,embryo_unit_huntington,embryo_Name_BlastExpandLast,embryo_Time_BlastExpandLast,embryo_Timestamp_BlastExpandLast,embryo_Value_BlastExpandLast,embryo_Name_BlastomereSize,embryo_Time_BlastomereSize,embryo_Timestamp_BlastomereSize,embryo_Value_BlastomereSize,embryo_Name_EVEN2,embryo_Time_EVEN2,embryo_Timestamp_EVEN2,embryo_Value_EVEN2,embryo_Name_EVEN4,embryo_Time_EVEN4,embryo_Timestamp_EVEN4,embryo_Value_EVEN4,embryo_Name_EVEN8,embryo_Time_EVEN8,embryo_Timestamp_EVEN8,embryo_Value_EVEN8,embryo_Name_FRAG2,embryo_Time_FRAG2,embryo_Timestamp_FRAG2,embryo_Value_FRAG2,embryo_Name_FRAG2CAT,embryo_Time_FRAG2CAT,embryo_Timestamp_FRAG2CAT,embryo_Value_FRAG2CAT,embryo_Name_FRAG4,embryo_Time_FRAG4,embryo_Timestamp_FRAG4,embryo_Value_FRAG4,embryo_Name_FRAG8,embryo_Time_FRAG8,embryo_Timestamp_FRAG8,embryo_Value_FRAG8,embryo_Name_Fragmentation,embryo_Time_Fragmentation,embryo_Timestamp_Fragmentation,embryo_Value_Fragmentation,embryo_Name_ICM,embryo_Time_ICM,embryo_Timestamp_ICM,embryo_Value_ICM,embryo_Name_Line,embryo_Time_Line,embryo_Timestamp_Line,embryo_Value_Line,embryo_Name_MN2Type,embryo_Time_MN2Type,embryo_Timestamp_MN2Type,embryo_Value_MN2Type,embryo_Name_MorphologicalGrade,embryo_Time_MorphologicalGrade,embryo_Timestamp_MorphologicalGrade,embryo_Value_MorphologicalGrade,embryo_Name_MultiNucleation,embryo_Time_MultiNucleation,embryo_Timestamp_MultiNucleation,embryo_Value_MultiNucleation,embryo_Name_Nuclei2,embryo_Time_Nuclei2,embryo_Timestamp_Nuclei2,embryo_Value_Nuclei2,embryo_Name_PN,embryo_Time_PN,embryo_Timestamp_PN,embryo_Value_PN,embryo_Name_Pulsing,embryo_Time_Pulsing,embryo_Timestamp_Pulsing,embryo_Value_Pulsing,embryo_Name_ReexpansionCount,embryo_Time_ReexpansionCount,embryo_Timestamp_ReexpansionCount,embryo_Value_ReexpansionCount,embryo_Name_Strings,embryo_Time_Strings,embryo_Timestamp_Strings,embryo_Value_Strings,embryo_Name_TE,embryo_Time_TE,embryo_Timestamp_TE,embryo_Value_TE,embryo_Name_t2,embryo_Time_t2,embryo_Timestamp_t2,embryo_Value_t2,embryo_Name_t3,embryo_Time_t3,embryo_Timestamp_t3,embryo_Value_t3,embryo_Name_t4,embryo_Time_t4,embryo_Timestamp_t4,embryo_Value_t4,embryo_Name_t5,embryo_Time_t5,embryo_Timestamp_t5,embryo_Value_t5,embryo_Name_t6,embryo_Time_t6,embryo_Timestamp_t6,embryo_Value_t6,embryo_Name_t7,embryo_Time_t7,embryo_Timestamp_t7,embryo_Value_t7,embryo_Name_t8,embryo_Time_t8,embryo_Timestamp_t8,embryo_Value_t8,embryo_Name_t9,embryo_Time_t9,embryo_Timestamp_t9,embryo_Value_t9,embryo_Name_tB,embryo_Time_tB,embryo_Timestamp_tB,embryo_Value_tB,embryo_Name_tEB,embryo_Time_tEB,embryo_Timestamp_tEB,embryo_Value_tEB,embryo_Name_tHB,embryo_Time_tHB,embryo_Timestamp_tHB,embryo_Value_tHB,embryo_Name_tM,embryo_Time_tM,embryo_Timestamp_tM,embryo_Value_tM,embryo_Name_tPB2,embryo_Time_tPB2,embryo_Timestamp_tPB2,embryo_Value_tPB2,embryo_Name_tPNa,embryo_Time_tPNa,embryo_Timestamp_tPNa,embryo_Value_tPNa,embryo_Name_tPNf,embryo_Time_tPNf,embryo_Timestamp_tPNf,embryo_Value_tPNf,embryo_Name_tSB,embryo_Time_tSB,embryo_Timestamp_tSB,embryo_Value_tSB,embryo_Name_tSC,embryo_Time_tSC,embryo_Timestamp_tSC,embryo_Value_tSC,embryo_PatientIDx,embryo_TreatmentName


### 9. Embryoscope Clinisys Combined


In [77]:
print("=== EMBRYOSCOPE CLINISYS COMBINED DATA ===")
combined_data = results['embryoscope_clinisys_combined']
print(f"Records: {len(combined_data)}")
if not combined_data.empty:
    display(combined_data.head(10))
else:
    print("No embryoscope clinisys combined data found.")


=== EMBRYOSCOPE CLINISYS COMBINED DATA ===
Records: 12


,oocito_id,oocito_id_micromanipulacao,oocito_diaseguinte,oocito_Maturidade,oocito_Embriologista,oocito_PI,oocito_TCD,oocito_PGD,oocito_ResultadoPGD,oocito_IdentificacaoPGD,oocito_Fertilizacao,oocito_CorpusculoPolar,oocito_ComentariosDepois,oocito_OocitoDoado,oocito_OrigemOocito,oocito_InseminacaoOocito,oocito_NClivou_D3,oocito_NCelulas_D3,oocito_Frag_D3,oocito_Blastomero_D3,oocito_NClivou_D4,oocito_NCelulas_D4,oocito_Compactando_D4,oocito_MassaInterna_D4,oocito_Trofoblasto_D4,oocito_NClivou_D5,oocito_NCelulas_D5,oocito_Compactando_D5,oocito_MassaInterna_D5,oocito_Trofoblasto_D5,oocito_NClivou_D6,oocito_NCelulas_D6,oocito_Compactando_D6,oocito_MassaInterna_D6,oocito_Trofoblasto_D6,oocito_NClivou_D7,oocito_NCelulas_D7,oocito_Compactando_D7,oocito_MassaInterna_D7,oocito_Trofoblasto_D7,oocito_score_maia,oocito_relatorio_ia,oocito_flag_embryoscope,oocito_embryo_number,micro_codigo_ficha,micro_numero_caso,micro_prontuario,micro_IdadeEsposa_DG,micro_IdadeMarido_DG,micro_Data_DL,micro_horario_inicial_fert,micro_horario_final_fert,micro_Aspiracao_DL,micro_medico_id,micro_responsavel_puncao_id,micro_recepcao_ovulos,micro_Denud_DL,micro_checagem_fertilizacao,micro_checagem_d3,micro_checagem_d5,micro_checagem_d6,micro_checagem_d7,micro_foliculos,micro_oocitos,micro_ICSIDescongelados,micro_pgd_oocito,micro_pgd_d5,micro_data_procedimento,micro_tecnica_injetados,micro_motilidade,micro_morfologia_injetados,micro_gradiente_descontinuo,micro_unidade_id,micro_responsavel_denud_id,micro_profissional_fertilizacao_id,micro_profissional_d3_id,micro_profissional_d5_id,micro_profissional_d6_id,micro_profissional_d7_id,micro_materiais_insumos,micro_local_coleta,micro_viscosidade,micro_volume_amostra_utilizada,micro_morfologia_pre,micro_morfologia_pos,micro_zymot,micro_observacoes_resumo_ciclos,micro_agulha_icsi,micro_lote_agulha_icsi,micro_validade_agulha_icsi,micro_agulha_aspir,micro_lote_agulha_aspir,micro_validade_agulha_aspir,micro_incubadora,micro_abstinencia_dias,micro_meios,micro_responsavel_oocitos_id,micro_fonte,micro_inicio_ps,micro_termino_ps,micro_volume_ps1,micro_CongelamentoOvulos,micro_concentracao_progressivos_pre,micro_conc_ps1,micro_EstadoSptz,micro_check_gradiente_ps,micro_check_swimup,micro_volume_ps2,micro_concentracao_progressivos_pos,micro_conc_ps2,micro_comentarios_ps,micro_responsavel_ps,micro_timelapse,micro_transferidos,micro_Transfer_D5,micro_medico_responsavel_id,micro_azoospermia,micro_Observacoes,micro_responsavel_labfiv,micro_responsavel_labfiv_data,micro_tipo_paciente,micro_alteracoes_oocitarias_normal,micro_alteracoes_oocitarias_granulacao,micro_alteracoes_oocitarias_pelucida,micro_alteracoes_oocitarias_oval,micro_alteracoes_oocitarias_inclusoes,micro_alteracoes_oocitarias_pervitelineo_aumentado,micro_alteracoes_oocitarias_irregular,micro_alteracoes_oocitarias_necrotica,micro_alteracoes_oocitarias_pervitelineo_granular,micro_alteracoes_oocitarias_ser,micro_alteracoes_oocitarias_corpusculo_frag,micro_alteracoes_oocitarias_vacuolos,micro_alteracoes_oocitarias_corpusculo_aum,micro_alteracoes_oocitarias_membrana,micro_alteracoes_oocitarias_corpusculo_peq,micro_alteracoes_oocitarias_citoplasma,micro_alteracoes_oocitarias_corpusculo_deg,micro_controle_anual,micro_maia,cong_em_id,cong_em_CodCongelamento,cong_em_Unidade,cong_em_prontuario,cong_em_paciente,cong_em_Data,cong_em_Hora,cong_em_Ciclo,cong_em_CicloRecongelamento,cong_em_NEmbrioes,cong_em_NPailletes,cong_em_Identificacao,cong_em_Tambor,cong_em_Cane,cong_em_Cane2,cong_em_Tecnica,cong_em_D5,cong_em_D6,cong_em_Observacoes,cong_em_BiologoFIV,cong_em_BiologoFIV2,cong_em_status_financeiro,descong_em_id,descong_em_CodDescongelamento,descong_em_Unidade,descong_em_prontuario,descong_em_doadora,descong_em_DataCongelamento,descong_em_DataDescongelamento,descong_em_Ciclo,descong_em_Identificacao,descong_em_CodCongelamento,descong_em_Tambor,descong_em_Cane,descong_em_PailletesDescongeladas,descong_em_Tecnica,descong_em_DataTransferencia,descong_em_Incubadora,descong_em_transferido

In [78]:
colunas = [
    'oocito_id','oocito_id_micromanipulacao', 'oocito_embryo_number', 'oocito_TCD', 
    'micro_codigo_ficha', 'micro_numero_caso', 'micro_Data_DL', 'micro_data_procedimento', 
    'cong_em_id', 'cong_em_CodCongelamento', 'cong_em_Ciclo', 'cong_em_NEmbrioes', 
    'treatment_TreatmentName', 'embryo_EmbryoID', 'embryo_KIDScore', 'embryo_Description', 'embryo_EmbryoDescriptionID', 'embryo_embryo_number',
    'emb_cong_transferidos', 'emb_cong_id', 'emb_cong_id_oocito', 'emb_cong_id_congelamento', 'emb_cong_id_descongelamento','emb_cong_embriao', 
    'descong_em_id', 'descong_em_transferidos_transferencia'	, 'descong_em_CodDescongelamento', 'descong_em_Ciclo' , 'descong_em_DataDescongelamento', 'descong_em_DataTransferencia', 
    'trat_id', 'trat_data_procedimento', 'trat_data_transferencia', 'trat_resultado_tratamento', 'trat_tentativa'
]

# combined_data.loc[combined_data['oocito_id_micromanipulacao'].isin([21961]), colunas]
# combined_data.loc[combined_data['embryo_EmbryoDescriptionID'].isin(['AA5']), colunas] 
combined_data.loc[:, colunas]

,oocito_id,oocito_id_micromanipulacao,oocito_embryo_number,oocito_TCD,micro_codigo_ficha,micro_numero_caso,micro_Data_DL,micro_data_procedimento,cong_em_id,cong_em_CodCongelamento,cong_em_Ciclo,cong_em_NEmbrioes,treatment_TreatmentName,embryo_EmbryoID,embryo_KIDScore,embryo_Description,embryo_EmbryoDescriptionID,embryo_embryo_number,emb_cong_transferidos,emb_cong_id,emb_cong_id_oocito,emb_cong_id_congelamento,emb_cong_id_descongelamento,emb_cong_embriao,descong_em_id,descong_em_transferidos_transferencia,descong_em_CodDescongelamento,descong_em_Ciclo,descong_em_DataDescongelamento,descong_em_DataTransferencia,trat_id,trat_data_procedimento,trat_data_transferencia,trat_resultado_tratamento,trat_tentativa
0,292126,28918,1,Descartado,28918,4575/25,2025-12-13,2025-12-13,<NA>,None,None,<NA>,2025-2849,D2025.12.13_S04488_I3027_P-1,NA,None,AA1,1,None,<NA>,<NA>,<NA>,<NA>,None,<NA>,None,None,None,NaT,NaT,40152,2025-12-13,2025-12-18,Gestação Clínica,1
1,292127,28918,2,Descartado,28918,4575/25,2025-12-13,2025-12-13,<NA>,None,None,<NA>,2025-2849,D2025.12.13_S04488_I3027_P-2,NA,None,AA2,2,None,<NA>,<NA>,<NA>,<NA>,None,<NA>,None,None,None,NaT,NaT,40152,2025-12-13,2025-12-18,Gestação Clínica,1
2,292128,28918,3,Criopreservado,28918,4575/25,2025-12-13,2025-12-13,31177,E3487/25,4575/25,1,2025-2849,D2025.12.13_S04488_I3027_P-3,NA,None,AA3,3,None,93818,292128,31177,0,(3),<NA>,None,None,None,NaT,NaT,40152,2025-12-13,2025-12-18,Gestação Clínica,1
3,292129,28918,4,Descartado,28918,4575/25,2025-12-13,2025-12-13,<NA>,None,None,<NA>,2025-2849,D2025.12.13_S04488_I3027_P-4,NA,None,AA4,4,None,<NA>,<NA>,<NA>,<NA>,None,<NA>,None,None,None,NaT,NaT,40152,2025-12-13,2025-12-18,Gestação Clínica,1
4,292130,28918,5,Descartado,28918,4575/25,2025-12-13,2025-12-13,<NA>,None,None,<NA>,2025-2849,D2025.12.13_S04488_I3027_P-5,NA,None,AA5,5,None,<NA>,<NA>,<NA>,<NA>,None,<NA>,None,None,None,NaT,NaT,40152,2025-12-13,2025-12-18,Gestação Clínica,1
5,292131,28918,6,Descartado,28918,4575/25,2025-12-13,2025-12-13,<NA>,None,None,<NA>,2025-2849,D2025.12.13_S04488_I3027_P-6,NA,None,AA6,6,None,<NA>,<NA>,<NA>,<NA>,None,<NA>,None,None,None,NaT,NaT,40152,2025-12-13,2025-12-18,Gestação Clínica,1
6,292132,28918,7,Descartado,28918,4575/25,2025-12-13,2025-12-13,<NA>,None,None,<NA>,2025-2849,D2025.12.13_S04488_I3027_P-7,NA,None,AA7,7,None,<NA>,<NA>,<NA>,<NA>,None,<NA>,None,None,None,NaT,NaT,40152,2025-12-13,2025-12-18,Gestação Clínica,1
7,292133,28918,8,Descartado,28918,4575/25,2025-12-13,2025-12-13,<NA>,None,None,<NA>,2025-2849,D2025.12.13_S04488_I3027_P-8,NA,None,AA8,8,None,<NA>,<NA>,<NA>,<NA>,None,<NA>,None,None,None,NaT,NaT,40152,2025-12-13,2025-12-18,Gestação Clínica,1
8,292134,28918,9,Transferido,28918,4575/25,2025-12-13,2025-12-13,<NA>,None,None,<NA>,2025-2849,D2025.12.13_S04488_I3027_P-9,NA,None,AA9,9,None,<NA>,<NA>,<NA>,<NA>,None,<NA>,None,None,None,NaT,NaT,40152,2025-12-13,2025-12-18,Gestação Clínica,1
9,292135,28918,10,Descartado,28918,4575/25,2025-12-13,2025-12-13,<NA>,None,None,<NA>,2025-2849,D2025.12.13_S04488_I3027_P-10,NA,None,AA10,10,None,<NA>,<NA>,<NA>,<NA>,None,<NA>,None,None,None,NaT,NaT,40152,2025-12-13,2025-12-18,Gestação Clínica,1


### 11. Planilha Embriologia


In [79]:
print("=== PLANILHA EMBRIOLOGIA DATA ===")
planilha_data = results['planilha_embriologia']
print(f"Records: {len(planilha_data)}")
if not planilha_data.empty:
    display(planilha_data)
else:
    print("No planilha embriologia data found.")


=== PLANILHA EMBRIOLOGIA DATA ===
Records: 0
No planilha embriologia data found.


### 12. Congelamentos Ovulos


In [80]:
print("=== CONGELAMENTOS OVULOS DATA ===")
congelamentos_ovulos_data = results['congelamentos_ovulos']
print(f"Records: {len(congelamentos_ovulos_data)}")
if not congelamentos_ovulos_data.empty:
    display(congelamentos_ovulos_data)
else:
    print("No congelamentos ovulos data found.")


=== CONGELAMENTOS OVULOS DATA ===
Records: 1


,id,CodCongelamento,Unidade,prontuario,paciente,Data,Hora,Ciclo,NOvulos,NPailletes,Identificacao,Tambor,Cane,Cane2,Tecnica,Motivo,BiologoFIV,BiologoFIV2,status_financeiro,hash,extraction_timestamp
0,10419,EGG1962/17,1,150547,esposa,2017-06-11,1900-01-01 12:00:00,None,12,3,EGG1962,U,6,None,Vitrificação,None,Mariana Nicolielo,Mariana Nicolielo,None,ad4fbdeca06326a73cae26b3d775ff15,2025-07-21 21:38:49


### 13. Descongelamentos Ovulos


In [81]:
print("=== DESCONGELAMENTOS OVULOS DATA ===")
descongelamentos_ovulos_data = results['descongelamentos_ovulos']
print(f"Records: {len(descongelamentos_ovulos_data)}")
if not descongelamentos_ovulos_data.empty:
    display(descongelamentos_ovulos_data)
else:
    print("No descongelamentos ovulos data found.")


=== DESCONGELAMENTOS OVULOS DATA ===
Records: 1


,id,CodDescongelamento,Unidade,prontuario,doadora,DataCongelamento,DataDescongelamento,Ciclo,Identificacao,CodCongelamento,Tambor,Cane,PailletesDescongeladas,Tecnica,Observacoes,BiologoFIV,BiologoFIV2,hash,extraction_timestamp
0,5128,DO779/25,1,150547,0,2017-06-11,2025-12-13,-,EGG1962,EGG1962/17,U,6,3,Desvitrificação,None,Ana Reis,Bruna Lázaro Lourenço,f5f496c4cc6fc2bd51e877320583debc,2025-12-15 19:26:27


### 14. Ovulos Congelados


In [82]:
print("=== OVULOS CONGELADOS DATA ===")
ovulos_congelados_data = results['ovulos_congelados']
print(f"Records: {len(ovulos_congelados_data)}")
if not ovulos_congelados_data.empty:
    display(ovulos_congelados_data)
else:
    print("No ovulos congelados data found.")


=== OVULOS CONGELADOS DATA ===
Records: 12


,id,id_congelamento,id_descongelamento,prontuario,pailletes,ovulo,comentarios,dia2,maturidade,dia2_2,tanque_amostra,caneca_amostra,rack_amostra,hash,extraction_timestamp
0,64848,10419,5128,150547,P3,12.0,None,DEG,MII,None,None,None,None,5d6b718e7689cde08a5e40f8e7c22a9c,2025-12-15 19:29:35
1,64847,10419,5128,150547,P3,11.0,None,MII,MII,None,None,None,None,ae284db1c916b398226387b7e1a64e0c,2025-12-15 19:29:35
2,64846,10419,5128,150547,P3,10.0,None,MII,MII,None,None,None,None,95cdc7036a1a43e04e63028b6dabe2b1,2025-12-15 19:29:35
3,64845,10419,5128,150547,P3,9.0,None,MII,MII,None,None,None,None,89ac3edd8f8374d3ce4c0a87c2482187,2025-12-15 19:29:35
4,64844,10419,5128,150547,P2,8.0,None,MII,MII,None,None,None,None,42758a9644374d069de56cefb6f47d1d,2025-12-15 19:29:35
5,64843,10419,5128,150547,P2,7.0,None,MII,MII,None,None,None,None,15a22ee81accbd36f8f2d4a3e26d8864,2025-12-15 19:29:35
6,64842,10419,5128,150547,P2,6.0,None,MII,MII,None,None,None,None,2b3c539d471891473034fb68dfe9ef68,2025-12-15 19:29:35
7,64841,10419,5128,150547,P2,5.0,None,MII,MII,None,None,None,None,476160732cc6aab9b7fc088f8d949cf1,2025-12-15 19:29:35
8,64840,10419,5128,150547,P1,4.0,None,MII,MII,None,None,None,None,84db9c1c9d512827f70b2313cba498a8,2025-12-15 19:29:35
9,64839,10419,5128,150547,P1,3.0,None,MII,MII,None,None,None,None,bc1b0c900958da26e602371e9b4ff7b0,2025-12-15 19:29:35


### 12. Planilha Embryoscope Combined


In [83]:
print("=== PLANILHA EMBRYOSCOPE COMBINED DATA ===")
combined_planilha_data = results['planilha_embryoscope_combined']
print(f"Records: {len(combined_planilha_data)}")
if not combined_planilha_data.empty:
    display(combined_planilha_data)
else:
    print("No planilha embryoscope combined data found.")


=== PLANILHA EMBRYOSCOPE COMBINED DATA ===
Records: 12


,oocito_id,oocito_id_micromanipulacao,oocito_diaseguinte,oocito_Maturidade,oocito_Embriologista,oocito_PI,oocito_TCD,oocito_PGD,oocito_ResultadoPGD,oocito_IdentificacaoPGD,oocito_Fertilizacao,oocito_CorpusculoPolar,oocito_ComentariosDepois,oocito_OocitoDoado,oocito_OrigemOocito,oocito_InseminacaoOocito,oocito_NClivou_D3,oocito_NCelulas_D3,oocito_Frag_D3,oocito_Blastomero_D3,oocito_NClivou_D4,oocito_NCelulas_D4,oocito_Compactando_D4,oocito_MassaInterna_D4,oocito_Trofoblasto_D4,oocito_NClivou_D5,oocito_NCelulas_D5,oocito_Compactando_D5,oocito_MassaInterna_D5,oocito_Trofoblasto_D5,oocito_NClivou_D6,oocito_NCelulas_D6,oocito_Compactando_D6,oocito_MassaInterna_D6,oocito_Trofoblasto_D6,oocito_NClivou_D7,oocito_NCelulas_D7,oocito_Compactando_D7,oocito_MassaInterna_D7,oocito_Trofoblasto_D7,oocito_score_maia,oocito_relatorio_ia,oocito_flag_embryoscope,oocito_embryo_number,micro_codigo_ficha,micro_numero_caso,micro_prontuario,micro_IdadeEsposa_DG,micro_IdadeMarido_DG,micro_Data_DL,micro_horario_inicial_fert,micro_horario_final_fert,micro_Aspiracao_DL,micro_medico_id,micro_responsavel_puncao_id,micro_recepcao_ovulos,micro_Denud_DL,micro_checagem_fertilizacao,micro_checagem_d3,micro_checagem_d5,micro_checagem_d6,micro_checagem_d7,micro_foliculos,micro_oocitos,micro_ICSIDescongelados,micro_pgd_oocito,micro_pgd_d5,micro_data_procedimento,micro_tecnica_injetados,micro_motilidade,micro_morfologia_injetados,micro_gradiente_descontinuo,micro_unidade_id,micro_responsavel_denud_id,micro_profissional_fertilizacao_id,micro_profissional_d3_id,micro_profissional_d5_id,micro_profissional_d6_id,micro_profissional_d7_id,micro_materiais_insumos,micro_local_coleta,micro_viscosidade,micro_volume_amostra_utilizada,micro_morfologia_pre,micro_morfologia_pos,micro_zymot,micro_observacoes_resumo_ciclos,micro_agulha_icsi,micro_lote_agulha_icsi,micro_validade_agulha_icsi,micro_agulha_aspir,micro_lote_agulha_aspir,micro_validade_agulha_aspir,micro_incubadora,micro_abstinencia_dias,micro_meios,micro_responsavel_oocitos_id,micro_fonte,micro_inicio_ps,micro_termino_ps,micro_volume_ps1,micro_CongelamentoOvulos,micro_concentracao_progressivos_pre,micro_conc_ps1,micro_EstadoSptz,micro_check_gradiente_ps,micro_check_swimup,micro_volume_ps2,micro_concentracao_progressivos_pos,micro_conc_ps2,micro_comentarios_ps,micro_responsavel_ps,micro_timelapse,micro_transferidos,micro_Transfer_D5,micro_medico_responsavel_id,micro_azoospermia,micro_Observacoes,micro_responsavel_labfiv,micro_responsavel_labfiv_data,micro_tipo_paciente,micro_alteracoes_oocitarias_normal,micro_alteracoes_oocitarias_granulacao,micro_alteracoes_oocitarias_pelucida,micro_alteracoes_oocitarias_oval,micro_alteracoes_oocitarias_inclusoes,micro_alteracoes_oocitarias_pervitelineo_aumentado,micro_alteracoes_oocitarias_irregular,micro_alteracoes_oocitarias_necrotica,micro_alteracoes_oocitarias_pervitelineo_granular,micro_alteracoes_oocitarias_ser,micro_alteracoes_oocitarias_corpusculo_frag,micro_alteracoes_oocitarias_vacuolos,micro_alteracoes_oocitarias_corpusculo_aum,micro_alteracoes_oocitarias_membrana,micro_alteracoes_oocitarias_corpusculo_peq,micro_alteracoes_oocitarias_citoplasma,micro_alteracoes_oocitarias_corpusculo_deg,micro_controle_anual,micro_maia,cong_em_id,cong_em_CodCongelamento,cong_em_Unidade,cong_em_prontuario,cong_em_paciente,cong_em_Data,cong_em_Hora,cong_em_Ciclo,cong_em_CicloRecongelamento,cong_em_NEmbrioes,cong_em_NPailletes,cong_em_Identificacao,cong_em_Tambor,cong_em_Cane,cong_em_Cane2,cong_em_Tecnica,cong_em_D5,cong_em_D6,cong_em_Observacoes,cong_em_BiologoFIV,cong_em_BiologoFIV2,cong_em_status_financeiro,descong_em_id,descong_em_CodDescongelamento,descong_em_Unidade,descong_em_prontuario,descong_em_doadora,descong_em_DataCongelamento,descong_em_DataDescongelamento,descong_em_Ciclo,descong_em_Identificacao,descong_em_CodCongelamento,descong_em_Tambor,descong_em_Cane,descong_em_PailletesDescongeladas,descong_em_Tecnica,descong_em_DataTransferencia,descong_em_Incubadora,descong_em_transferido